WITH 
all_users AS (
    SELECT st_id, test_grp FROM studs
),

active_users AS (
    SELECT st_id 
    FROM peas
    WHERE correct = true
    GROUP BY st_id
    HAVING COUNT(*) > 10
),


math_active_users AS (
    SELECT st_id 
    FROM peas
    WHERE correct = true AND subject = 'Math'
    GROUP BY st_id
    HAVING COUNT(*) >= 2
),

paying_users AS (
    SELECT 
        st_id, 
        SUM(money) AS total_spent
    FROM final_project_check
    GROUP BY st_id
),


math_paying_users AS (
    SELECT DISTINCT st_id
    FROM final_project_check
    WHERE subject = 'Math'
)

SELECT 
    s.test_grp,
    -- ARPU (общий доход / все пользователи)
    COALESCE(SUM(p.total_spent), 0) / NULLIF(COUNT(DISTINCT s.st_id), 0) AS ARPU,
    
    -- ARPAU (общий доход / активные пользователи)
    COALESCE(SUM(p.total_spent), 0) / NULLIF(COUNT(DISTINCT a.st_id), 0) AS ARPAU,
    
    -- CR в покупку (покупатели / все пользователи) в десятичном формате
    CASE WHEN COUNT(DISTINCT s.st_id) > 0 
         THEN COUNT(DISTINCT p.st_id) * 1.0 / COUNT(DISTINCT s.st_id)
         ELSE 0 END AS CR,
    
    -- CR активного пользователя в покупку (активные покупатели / активные пользователи) в десятичном формате
    CASE WHEN COUNT(DISTINCT a.st_id) > 0
         THEN COUNT(DISTINCT CASE WHEN a.st_id IS NOT NULL THEN p.st_id END) * 1.0 / 
              COUNT(DISTINCT a.st_id)
         ELSE 0 END AS CR_active,
    
    -- CR пользователя из активности по математике в покупку курса по математике в десятичном формате
    CASE WHEN COUNT(DISTINCT m.st_id) > 0
         THEN COUNT(DISTINCT CASE WHEN m.st_id IS NOT NULL THEN mp.st_id END) * 1.0 / 
              COUNT(DISTINCT m.st_id)
         ELSE 0 END AS CR_active_math
FROM 
    all_users s
LEFT JOIN 
    paying_users p ON s.st_id = p.st_id
LEFT JOIN 
    active_users a ON s.st_id = a.st_id
LEFT JOIN 
    math_active_users m ON s.st_id = m.st_id
LEFT JOIN 
    math_paying_users mp ON s.st_id = mp.st_id
GROUP BY 
    s.test_grp;